In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-18 22:03:29 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-18 22:03:29 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-18 22:03:29 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-18 22:03:32,002] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-18 22:03:32 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpj2vaan5e/test.c -o /tmp/tmpj2vaan5e/test.o
2025-07-18 22:03:32 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpj2vaan5e/test.o -laio -o /tmp/tmpj2vaan5e/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-18 22:03:32 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmp2tui03bs/test.c -o /tmp/tmp2tui03bs/test.o
2025-07-18 22:03:32 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmp2tui03bs/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmp2tui03bs/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-18 22:03:33 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-18 22:03:33 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-18 22:03:33 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-18 22:03:33 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-18 22:03:34 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-18 22:03:34 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-07-18 22:03:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-18 22:03:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-18 22:04:24 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-18 22:04:24 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [ ]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-18 22:04:38 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])


2025-07-18 22:04:38 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-18 22:04:38 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-18 22:04:38 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-18 22:04:38 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-18 22:04:38 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-18 22:04:39 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-18 22:04:39 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-

## Generate samples and filter by LM knowledge

In [ ]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)

2025-07-18 22:02:23 src.selection.data INFO     Loaded 16 categories


In [ ]:
from src.selection.data import SelectionSample, get_random_sample

sample = get_random_sample(
    people_by_category = people_by_category,
    mt = mt,
    n_distractors=5,
    get_alt_obj=False,
    # category="actor",
    obj_idx=3,
    filter_by_lm_prediction=True
)
print(sample)
print(sample.prompt)
sample.prediction

2025-07-18 22:02:26 src.selection.data ERROR    Sample = Oprah Winfrey -> Marc Benioff (3): ['Aaron Donald', 'Dmitry Bivol', 'Ridley Scott', 'Marc Benioff', 'Vera Wang', 'Anthony Davis']
Top prediction " Vera"[68200] (p=0.355, logit=18.250) does not match the object Marc Benioff[19412, " Marc"].
Retrying ...

2025-07-18 22:02:26 src.selection.data ERROR    Sample = Reid Hoffman -> Oprah Winfrey (3): ['Marco Rubio', 'Micah Parsons', 'Ricky Gervais', 'Oprah Winfrey', 'Devin Haney', 'Brooks Koepka']
Top prediction " None"[2290] (p=0.342, logit=17.500) does not match the object Oprah Winfrey[92258, " Oprah"].
Retrying ...

Jeff Bezos -> Richard Branson (3): ['Guillermo del Toro', 'Jessica Pegula', 'Shakur Stevenson', 'Richard Branson', 'Gabriel Iglesias', 'Julia Roberts']
Which person from the following list has the profession in common with Jeff Bezos?
Options: Guillermo del Toro, Jessica Pegula, Shakur Stevenson, Richard Branson, Gabriel Iglesias, Julia Roberts.
Ans:


[PredictedToken(token=' Richard', prob=0.82421875, logit=19.75, token_id=12131, metadata=None),
 PredictedToken(token=' Jeff', prob=0.0361328125, logit=16.625, token_id=12149, metadata=None),
 PredictedToken(token=' The', prob=0.031982421875, logit=16.5, token_id=578, metadata=None),
 PredictedToken(token=' None', prob=0.0281982421875, logit=16.375, token_id=2290, metadata=None),
 PredictedToken(token=' R', prob=0.009765625, logit=15.3125, token_id=432, metadata=None)]

In [ ]:
##########################################################
save_dir = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR, "selection", mt.name.split("/")[-1], "profession"
)
os.makedirs(save_dir, exist_ok=True)
file_name = "filtered_samples.json"

LIMIT = 12000
N_DISTRACTORS = 5
SAVE_STEP = 100
##########################################################

from src.utils.experiment_utils import set_seed
set_seed(123456)

filtered_samples = []
while len(filtered_samples) < LIMIT:
    sample = get_random_sample(
        people_by_category=people_by_category,
        mt=mt,
        n_distractors=N_DISTRACTORS,
        get_alt_obj=False,
        filter_by_lm_prediction=True
    )
    sample.detensorize()
    filtered_samples.append(sample)
    print(f"Collected {len(filtered_samples)}/{LIMIT} samples. {len(filtered_samples) / LIMIT * 100:.2f}%")

    if len(filtered_samples) % SAVE_STEP == 0 or len(filtered_samples) == LIMIT:
        print(f"Saving {len(filtered_samples)} samples to {os.path.join(save_dir, file_name)}")
        with open(os.path.join(save_dir, file_name), "w") as f:
            json.dump([s.to_dict() for s in filtered_samples], f, indent=2)

# with open(os.path.join(save_dir, file_name), "w") as f:
#     json.dump([s.to_dict() for s in filtered_samples], f, indent=2)

2025-07-18 22:02:39 src.utils.experiment_utils INFO     setting all seeds to 123456
Collected 1/10000 samples. 0.01%
Collected 2/10000 samples. 0.02%
2025-07-18 22:02:41 src.selection.data ERROR    Sample = Oprah Winfrey -> Warren Buffett (4): ['Pete Davidson', 'Jalen Hurts', 'Taylor Swift', 'Ralph Lauren', 'Warren Buffett', 'Paulo Dybala']
Top prediction " Taylor"[16844] (p=0.469, logit=18.500) does not match the object Warren Buffett[26713, " Warren"].
Retrying ...

2025-07-18 22:02:41 src.selection.data ERROR    Sample = Richard Branson -> Oprah Winfrey (3): ['Bo Burnham', 'Davante Adams', 'Jimmy Butler', 'Oprah Winfrey', 'Nicholas Sparks', 'Mohamed Salah']
Top prediction " None"[2290] (p=0.316, logit=18.125) does not match the object Oprah Winfrey[92258, " Oprah"].
Retrying ...

Collected 3/10000 samples. 0.03%
Collected 4/10000 samples. 0.04%
Collected 5/10000 samples. 0.05%
Collected 6/10000 samples. 0.06%
Collected 7/10000 samples. 0.07%
Collected 8/10000 samples. 0.08%
Collecte

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f9b8419d590>>
Traceback (most recent call last):
  File "/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Collected 35/10000 samples. 0.35%
Collected 36/10000 samples. 0.36%
2025-07-18 22:03:13 src.selection.data ERROR    Sample = Sebastian Maniscalco -> Amy Schumer (5): ['Alexander Zverev', 'Terence Crawford', 'Wes Anderson', 'Julia Roberts', 'Peter Baker', 'Amy Schumer']
Top prediction " None"[2290] (p=0.168, logit=17.250) does not match the object Amy Schumer[29793, " Amy"].
Retrying ...

2025-07-18 22:03:13 src.selection.data ERROR    Sample = Bill Burr -> Kevin Hart (5): ['Johnny Depp', 'LeBron James', 'Terence Crawford', 'Mitt Romney', 'Spike Lee', 'Kevin Hart']
Top prediction " Ter"[10335] (p=0.473, logit=19.125) does not match the object Kevin Hart[16768, " Kevin"].
Retrying ...

Collected 37/10000 samples. 0.37%
Collected 38/10000 samples. 0.38%
Collected 39/10000 samples. 0.39%
Collected 40/10000 samples. 0.40%
Collected 41/10000 samples. 0.41%
Collected 42/10000 samples. 0.42%
Collected 43/10000 samples. 0.43%
Collected 44/10000 samples. 0.44%
Collected 45/10000 samples. 0.45%
C

In [ ]:
with open(os.path.join(save_dir, "filtered_samples.json"), "r") as f:
    loaded_samples = json.load(f)
loaded_samples = [SelectionSample.from_dict(s) for s in loaded_samples]

In [ ]:
print(loaded_samples[0].prompt)

## Cache last token states for the generated samples

In [ ]:
with open(os.path.join(save_dir, "filtered_samples.json"), "r") as f:
    filtered_samples = json.load(f)
filtered_samples = [SelectionSample.from_dict(s) for s in filtered_samples]

len(filtered_samples)

In [ ]:
#######################################################################################
cache_dir = os.path.join(save_dir, "cached_states")

all_layers = (
    [mt.embedder_name]  # embeddings
    + mt.layer_names  # residual
    + [mt.mlp_module_name_format.format(i) for i in range(mt.n_layer)]  # mlp outputs
    + [mt.attn_module_name_format.format(i) for i in range(mt.n_layer)]  # attn outputs
)
TOKEN_POSITION = -1

locations = [(layer_name, TOKEN_POSITION) for layer_name in all_layers]
#######################################################################################
os.makedirs(cache_dir, exist_ok=True)

import numpy as np
from src.utils.typing import TokenizerOutput
from src.functional import get_hs, detensorize
from src.tokens import prepare_input

for idx, sample in enumerate(filtered_samples):
    # inputs = TokenizerOutput(data = sample.metadata["tokenized"]).to(mt.device)
    inputs = prepare_input(prompts=sample.prompt, tokenizer=mt)
    sample.detensorize()

    cache = {"sample": sample.to_dict(), "states": {}}
    states = get_hs(
        mt=mt,
        input=inputs,
        locations=locations,
        return_dict=True,
    )

    for (layer_name, tok_idx), state in states.items():
        cache["states"][layer_name] = state.detach().to(torch.float32).cpu().numpy()

    cache = detensorize(cache)
    np.savez_compressed(
        os.path.join(cache_dir, f"sample_{idx}.npz"), **cache, allow_pickle=True
    )

    logger.info(
        f"Processed sample {idx + 1}/{len(filtered_samples)} ({(idx + 1) / len(filtered_samples) * 100:.2f}%)"
    )

In [2]:
val = 5
print(f"{val:05d}")

00005


### Testing by loading the cached states

In [4]:
import os
import numpy as np

# cache_dir = os.path.join(save_dir, "cached_states")
cache_dir = "/disk/u/arnab/Codes/Projects/retrieval/results/selection/Llama-3.3-70B-Instruct/profession/cached_states/last_token/Llama-3.3-70B-Instruct"
os.listdir(cache_dir)

['sample_00009.npz',
 'sample_00006.npz',
 'sample_00004.npz',
 'sample_00007.npz',
 'sample_00008.npz',
 'sample_00000.npz',
 'sample_00001.npz',
 'sample_00002.npz',
 'sample_00005.npz',
 'sample_00003.npz']

In [11]:
sample_states = np.load(os.path.join(cache_dir, "sample_00001.npz"), allow_pickle=True)
sample_states.files

['sample', 'states']

In [12]:
from src.selection.data import SelectionSample

sample = SelectionSample.from_dict(sample_states["sample"].item())
print(sample.prompt)
print(sample.prediction)

[2025-07-18 15:22:19,475] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-18 15:22:19 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpclkskj1n/test.c -o /tmp/tmpclkskj1n/test.o
2025-07-18 15:22:19 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpclkskj1n/test.o -laio -o /tmp/tmpclkskj1n/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-18 15:22:19 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpk7_2kkp2/test.c -o /tmp/tmpk7_2kkp2/test.o
2025-07-18 15:22:19 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpk7_2kkp2/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpk7_2kkp2/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


Which person from the following list has the profession in common with Al Pacino?
Caroline Garcia, Dak Prescott, Justin Bieber, Ryan Reynolds, Wolf Blitzer, Dmitry Bivol.
Ans:
(PredictedToken(token=' Ryan', prob=0.5625, logit=18.625, token_id=13960, metadata=None), PredictedToken(token=' None', prob=0.1611328125, logit=17.375, token_id=2290, metadata=None), PredictedToken(token=' Wolf', prob=0.05224609375, logit=16.25, token_id=26296, metadata=None), PredictedToken(token=' Al', prob=0.05224609375, logit=16.25, token_id=1708, metadata=None), PredictedToken(token=' The', prob=0.028076171875, logit=15.625, token_id=578, metadata=None))


In [14]:
states = {
    layer_name: torch.Tensor(value)
    for layer_name, value in sample_states["states"].item().items()
}

states

{'model.embed_tokens_<>_-1': tensor([ 3.1233e-05,  2.3651e-03,  2.6855e-03,  ...,  8.6594e-04,
         -2.4567e-03,  3.0975e-03]),
 'model.layers.0_<>_-1': tensor([-0.0013,  0.0027,  0.0020,  ..., -0.0006, -0.0028, -0.0032]),
 'model.layers.1_<>_-1': tensor([-0.0051,  0.0013,  0.0033,  ...,  0.0009, -0.0060, -0.0009]),
 'model.layers.2_<>_-1': tensor([-0.0139, -0.0017, -0.0034,  ..., -0.0018,  0.0020, -0.0110]),
 'model.layers.3_<>_-1': tensor([-0.0219,  0.0034,  0.0067,  ..., -0.0047,  0.0046, -0.0030]),
 'model.layers.4_<>_-1': tensor([-0.0239, -0.0037,  0.0079,  ..., -0.0107,  0.0090, -0.0035]),
 'model.layers.5_<>_-1': tensor([-0.0228, -0.0071,  0.0060,  ...,  0.0010,  0.0206, -0.0059]),
 'model.layers.6_<>_-1': tensor([-0.0115,  0.0031,  0.0123,  ..., -0.0060,  0.0312, -0.0151]),
 'model.layers.7_<>_-1': tensor([-0.0222,  0.0012,  0.0287,  ...,  0.0160,  0.0131, -0.0059]),
 'model.layers.8_<>_-1': tensor([-0.0167,  0.0092,  0.0352,  ...,  0.0317,  0.0276,  0.0002]),
 'model.layer